In [1]:
import os
import re
import operator
from collections import Counter

File to read titles from:

In [2]:
datafile = "/home/ctroupin/ULg/gher-ulg.github.io/Liege-Colloquium/topics.md"
jsonfile = "../data/topicsWordle.js"
jsonfile2 = "/home/ctroupin/ULg/gher-ulg.github.io/data/topicsWordle.js"

In [3]:
topiclist = []
with open(datafile, "r") as f:
    for lines in f:
        # Split line with | as separator
        topic = lines.split("|")[2]
        # Convert to lower case
        topic = [x.lower() for x in topic.split()]
        # Remove final 's' (assuming it's plural)
        # topic = [x.rstrip()[:-1] if (x.rstrip()[-1] == "s") else x.rstrip() for x in topic]
        # Add to list
        topiclist.extend(topic)
# Convert list to string
wordstring = " ".join(topiclist)

## Remove punctuation and common words

In [4]:
punctuation = [":", ".", "–", '"', ",", "'"]

removelist = ["\n", "the", "in", "a", "of", "les",
              "from" "le", "à", "but",
              "and","to", "for", "at", "on", 
              "no", "specific", "topic", "identifiable", "contents",
              "several", "papers", "related", "easily"
              "contents"]

substdict = {"l’océan": "ocean",
              "from": " ",
              "équations": "equations",
              "mesoscale/synoptic": "mesoscale synoptic",
              "modeling": "modelling", "no specific topic": " ",
              "sub-mesoscale": "submesoscale", 
              "hydrodynamically": 'hydrodynamics', 
              "hydrodynamical": 'hydrodynamics', 
              "hydrodynamiques": 'hydrodynamics', 
              "hydrodynamic": 'hydrodynamics', 
              "seas": "sea", "changes": "change",
              "waters": "water", 
              "ecosystems": "ecosystem",
              "estuarine": "estuaries",
              "applicable": "applicable",
              "tracers": "tracer",
              "models": "modelling",
              "re³-visited": "revisited",
              "re-revisited": "revisited",
              "environmental": "environments",
              "remote sensing": "remote-sensing",
              "bio-geo-chemical": "biogeochemical",
              "analyses": "analysis",
              "interaction": "interactions",
              "prediction": "predictions"
               }

for symbols in punctuation:
    wordstring = wordstring.replace(symbols, ' ')
for words in removelist:
    wordstring = re.sub(r"\b{}\b".format(words), ' ', wordstring)
for oldvalue, newvalue in substdict.items():
    wordstring = re.sub(r"{}\b".format(oldvalue), newvalue, wordstring)

## Count word frequency and sort

In [5]:
worddict = Counter(wordstring.split())
sortedworddict = sorted(worddict.items(), key=operator.itemgetter(1), reverse=True)

# Convert the dictionary to json
We should get something like this: 
```
var words = [
  {text: 'have', size: 102},
  {text: 'Oliver', size: 47},
  {text: 'say', size: 46},
  {text: 'said', size: 36}
]
```

In [6]:
with open(jsonfile, "w") as jf:
    jf.write("var words = [\n")
    for lines in sortedworddict:
        jf.write("".join(("\t{text: '", lines[0], "', size: ", str(lines[1]), "},\n")))
    jf.write("]")

# Build dictionary with the years for each word

In [131]:
clqdict = {}
with open(datafile, "r") as f:
    for lines in f:
        # Split line with | as separator
        year = int(lines.split("|")[1])
        topic = lines.split("|")[2]
        
        for wordstring in topic.split():
            # Convert to lower case
            wordstring = wordstring.lower()
            
            for symbols in punctuation:
                wordstring = wordstring.replace(symbols, ' ')
            for words in removelist:
                wordstring = re.sub(r"\b{}\b".format(words), ' ', wordstring)
            for oldvalue, newvalue in substdict.items():
                wordstring = re.sub(r"{}\b".format(oldvalue), newvalue, wordstring)
            
            for w in wordstring.split():
                if w not in removelist:
                    if w in clqdict.keys():
                        clqdict[w].insert(0, year)
                    else:
                        clqdict[w] = [year]
clqdict

{'50': [2018],
 'air-sea': [1992],
 'analysis': [1999, 2010],
 'anthropogenic': [2011],
 'applicables': [1969],
 'arctic': [2008],
 'assimilation': [1993, 2001],
 'biochemical': [1971],
 'biogeochemical': [2001, 2011],
 'biological': [1990],
 'bottom': [1976],
 'celebration': [2018],
 'challenges': [2012],
 'change': [1994, 2008, 2011],
 'changing': [2008],
 'chemical': [1990],
 'circulation': [1999],
 'climate': [2008],
 'coastal': [1991, 1994, 2009],
 'coherent': [1988],
 'colloquium': [2018],
 'colour': [2012],
 'conditions': [2008],
 'continental': [1975],
 'coupled': [1984, 1989, 2001],
 'covered': [1990, 1998],
 'data': [1993, 2001],
 'dead': [2003],
 'deep': [1970, 1991],
 'diagnostic': [1999],
 'dominated': [1996],
 'dying': [2003],
 'dynamics': [1975, 1986, 2002],
 'easily': [1973, 1974],
 'ecohydrodynamics': [1980, 1985],
 'ecological': [2001],
 'ecosystem': [1996, 1998, 2006],
 'edges': [1990],
 'environments': [2004, 2014, 2015],
 'equations': [1969, 1971],
 'equatorial': [

## Convert dictionary to json file

In [132]:
with open(jsonfile2, "w") as jf:
    jf.write("var words = [\n")
    jf.write("\t{text: 'colloquium', size: 102, yearlist: '1969-2018'},\n")
    for k, v in clqdict.items():
        jf.write("".join(("\t{text: '", k, "', size: ", str(len(v)), ", yearlist: ", str(v),"},\n")))
    jf.write("]")
print("Written file {}".format(jsonfile2))

Written file /home/ctroupin/ULg/gher-ulg.github.io/data/topicsWordle.js


In [7]:
sortedworddict

[('ocean', 11),
 ('marine', 10),
 ('modelling', 10),
 ('hydrodynamics', 10),
 ('turbulence', 8),
 ('sea', 8),
 ('processes', 7),
 ('revisited', 3),
 ('environments', 3),
 ('estuaries', 3),
 ('water', 3),
 ('change', 3),
 ('coastal', 3),
 ('ecosystem', 3),
 ('dynamics', 3),
 ('coupled', 3),
 ('ice', 3),
 ('interactions', 3),
 ('shelf', 3),
 ('science', 2),
 ('submesoscale', 2),
 ('new', 2),
 ('monitoring', 2),
 ('predictions', 2),
 ('synoptic', 2),
 ('global', 2),
 ('scale', 2),
 ('remote-sensing', 2),
 ('tracer', 2),
 ('physical', 2),
 ('biogeochemical', 2),
 ('analysis', 2),
 ('influence', 2),
 ('geophysical', 2),
 ('data', 2),
 ('assimilation', 2),
 ('three-dimensional', 2),
 ('covered', 2),
 ('deep', 2),
 ('ocean-atmosphere', 2),
 ('mixing', 2),
 ('ecohydrodynamics', 2),
 ('easily', 2),
 ('equations', 2),
 ('long-term', 1),
 ('studies', 1),
 ('oceanography', 1),
 ('celebration', 1),
 ('50', 1),
 ('years', 1),
 ('liege', 1),
 ('colloquium', 1),
 ('mechanisms', 1),
 ('implications', 1